In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

C:\Users\Ryan Jewik\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Ryan Jewik\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# modeling imports
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV # Linear Regression Model
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, SplineTransformer, OneHotEncoder #Z-score variables, Polynomial
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error #model evaluation
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
# pipeline imports
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error #model evaluation
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut

In [3]:
# models
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor # Decision Tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor # random forest
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor # gradient boosting
import xgboost as xg
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [4]:
df = pd.read_csv('data\lotwize_case.csv')
valid_df = pd.read_csv('data\lotwize_case_validation-1.csv')

In [5]:
schools = pd.read_csv('data\school_data.csv')

In [6]:
schools.columns

Index(['OBJECTID', 'Academic Year', 'Fed ID', 'CDS Code', 'District Code',
       'School Code', 'Region', 'County Name', 'District Name', 'School Name',
       'School Type', 'Status', 'Open Date', 'Closed Date', 'School Level',
       'Grade Low', 'Grade High', 'Charter', 'Charter Num', 'Funding Type',
       'Virtual', 'Magnet', 'Title I', 'DASS', 'Assistance Status ESSA',
       'Street', 'City', 'Zip', 'State', 'US Congress District',
       'CA Senate District', 'CA Assembly District', 'Locale', 'Latitude',
       'Longitude', 'Enroll Total', 'African American', 'African American (%)',
       'American Indian', 'American Indian Pct', 'Asian', 'Asian (%)',
       'Filipino', 'Filipino (%)', 'Hispanic', 'Hispanic (%)',
       'Pacific Islander', 'Pacific Islander (%)', 'White', 'White (%)',
       'Two or More Races', 'Two or More Races (%)', 'Not Reported',
       'Not Reported (%)', 'English Learner', 'English Learner (%)', 'Foster',
       'Foster (%)', 'Homeless', 'Homeless (%)

In [7]:
predictors = [#'schools/1/rating',
#'schools/0/distance',
#'schools/0/name',
'mortgageRates/thirtyYearFixedRate',
#'nearbyHomes/1/price',
'bathrooms',
'homeType',
'adTargets/sqft',
'bedrooms',
'zipcode',
'yearBuilt',
'photoCount',
'longitude',
'latitude',
'city',
'price'
]

In [8]:
new_df = df[predictors]
new_df.shape

(9142, 12)

In [9]:
zipcodes = df[['zipcode', 'price']]

In [10]:
unique_schools_by_zipcode = schools.groupby('Zip')['School Name'].nunique().reset_index()
unique_schools_by_zipcode.columns = ['Zip', 'unique_school_count']
school_count = unique_schools_by_zipcode.rename(columns = {'Zip': 'zipcode', 'unique_school_count': 'school_count'})

In [11]:
new_df = pd.merge(new_df, school_count, on = 'zipcode')

In [12]:
new_df.shape

(9142, 13)

In [13]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two points on the Earth.
    
    Parameters:
    lat1, lon1: Latitude and Longitude of the first point.
    lat2, lon2: Latitude and Longitude of the second point.
    
    Returns:
    Distance between the two points in kilometers.
    """
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Radius of Earth in kilometers (mean radius)
    r = 6371.0
    
    # Calculate the distance
    distance = c * r
    
    return distance

In [14]:
def school_finder(row):
    zipcode = row['zipcode']
    home_lat = row['latitude']
    home_long = row['longitude']
    schools_in_area = schools[schools['Zip'] == zipcode][['School Name', 'Longitude', 'Latitude']]
    min_distance = 80000000000
    for school in schools_in_area['School Name']:
        school_lat = schools_in_area[schools_in_area['School Name'] == school]['Latitude']
        school_long = schools_in_area[schools_in_area['School Name'] == school]['Longitude']
        distance = haversine_distance(home_lat, home_long, school_lat, school_long)
        if distance < min_distance:
            min_distance = distance
    return min_distance

In [15]:
new_df.isnull().sum()

mortgageRates/thirtyYearFixedRate    7144
bathrooms                             531
homeType                              378
adTargets/sqft                       3166
bedrooms                              570
zipcode                                 0
yearBuilt                             566
photoCount                           1004
longitude                               9
latitude                              386
city                                  378
price                                   0
school_count                            0
dtype: int64

In [16]:
new_df = new_df.dropna().reset_index()
new_df.shape

(1593, 14)

In [17]:
import warnings
warnings.filterwarnings('ignore')
new_df['closest_school_distance'] = new_df.apply(school_finder, axis = 1)

In [18]:
new_df.shape

(1593, 15)

In [19]:
new_df = new_df.drop(columns = ['index', 'longitude', 'latitude'])


In [20]:
crime = pd.read_csv("data\crime_data\ca_offenses_by_city.csv")

In [21]:
crime.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition),Rape (legacy definition),Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson
0,Adelanto,"33,005",212,2,14,NaN,48,148,808,434,254,120,24
1,Agoura Hills,"20,970",15,0,1,NaN,6,8,310,82,217,11,0
2,Alameda,"78,613",148,2,7,NaN,61,78,"1,819",228,"1,245",346,18
3,Albany,"19,723",34,1,6,NaN,16,11,605,95,447,63,0
4,Alhambra3,"86,175",168,1,13,NaN,74,80,"1,929",305,"1,413",211,6


In [22]:
crime = crime[['Population', 'Violent crime', 'Property crime', 'City']]

In [23]:
#dividing the number of crimes reported to law enforcement by the total population and then multiplying by 100,000

In [24]:
def removeCommas(string):
    new_string = string.replace(",","")
    return new_string

In [25]:
crime['Population'] = crime['Population'].apply(removeCommas)
crime['Violent crime'] = crime['Violent crime'].apply(removeCommas)
crime['Property crime'] = crime['Property crime'].apply(removeCommas)

In [26]:
crime['Population'] = crime['Population'].astype(int)
crime['Violent crime'] = crime['Violent crime'].astype(int)
crime['Property crime'] = crime['Property crime'].astype(int)
crime.dtypes

Population         int32
Violent crime      int32
Property crime     int32
City              object
dtype: object

In [27]:
crime['crime rate'] = (((crime['Violent crime'] + crime['Property crime']) / crime['Population']) * 1000)

In [28]:
crime = crime.rename(columns = {'City': 'city'})
crime = crime[['city','crime rate']]

In [29]:
new_df = pd.merge(new_df, crime, on = 'city')

In [30]:
income = pd.read_csv('data\income\kaggle_income.csv',engine='python',encoding='latin1')

In [31]:
income = income[income['State_ab'] == 'CA']

In [32]:
income

,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
1588,6011848,6,California,CA,Modoc County,Bieber,Adin,CDP,place,96009,530,8901691,15858,41.199312,-120.956779,54602,300000,32641,37.898866
1589,6011858,6,California,CA,Humboldt County,Blocksburg,Alderpoint,CDP,place,95514,707,6288255,0,40.161347,-123.615729,24830,300000,14770,140.000000
1590,6011868,6,California,CA,San Diego County,Alpine,Alpine,CDP,place,91901,619,69370314,1475,32.842286,-116.756044,91662,79440,65636,2524.288055
1591,6011878,6,California,CA,Santa Cruz County,Watsonville,Amesti,CDP,place,95076,831,7747643,166234,36.959483,-121.781739,63761,48440,55214,620.110873
1592,6011888,6,California,CA,Santa Cruz County,Aptos,Aptos,CDP,place,95003,831,16657985,0,36.991147,-121.892758,96841,82737,67911,1074.376484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4863,6029969,6,California,CA,Alameda County,Richmond,Alameda city,Track,Track,94804,510,1959331,0,37.930196,-122.335027,71147,56532,61196,285.087188
4864,6029979,6,California,CA,Alameda County,El Cerrito,Alameda city,Track,Track,94530,510,1344741,0,37.909020,-122.311712,101488,83631,70848,233.383842
4865,6029989,6,California,CA,Alameda County,Richmond,Alameda city,Track,Track,94804,510,1019761,0,37.922571,-122.315517,61459,60522,40129,135.458549
4866,602999,6,California,CA,Alameda County,Bakersfield,Alameda city,Track,Track,93305,661,2350102,0,35.392270,-118.984646,30390,23044,25787,1853.290842


In [33]:
income = income.rename(columns = {'Zip_Code': 'zipcode', 'Median': 'Median Household Income'})
income = income[['zipcode', 'Median Household Income']]

In [34]:
new_df = pd.merge(new_df, income, on = 'zipcode')

In [35]:
new_df = new_df.drop(columns = ['city'])

In [36]:
unique_df = new_df.drop_duplicates()
unique_df.shape

(5024, 13)

In [37]:
predictors = new_df.columns.tolist()
contin = new_df.columns.tolist()
len(predictors)

13

In [38]:
predictors.remove('price')
contin.remove('homeType')
#contin.remove('schools/0/name')
contin.remove('zipcode')
contin.remove('price')
cat = ['homeType', 'zipcode']

In [39]:
new_df['zipcode'] = new_df["zipcode"].astype("category")
new_df['homeType'] = new_df["homeType"].astype("category")
#new_df['schools/0/name'] = new_df["schools/0/name"].astype("category")

In [40]:
len(predictors)

12

In [41]:
X = new_df[predictors]
y = new_df["price"]

In [42]:
contin = predictors
contin.remove('homeType')
contin.remove('zipcode')
contin

['mortgageRates/thirtyYearFixedRate',
 'bathrooms',
 'adTargets/sqft',
 'bedrooms',
 'yearBuilt',
 'photoCount',
 'school_count',
 'closest_school_distance',
 'crime rate',
 'Median Household Income']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 101)
xgb = xg.XGBRegressor(n_estimators = 1000, max_depth = 3, random_state = 101, booster = 'gbtree', learning_rate = 0.01, enable_categorical=True, 
                      eval_metric = "rmse", early_stopping_rounds=50, reg_lambda = 1, reg_alpha = 0.5)
xgb.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=True
)

# Make predictions and evaluate
y_pred = xgb.predict(X_test)
val_error = mean_squared_error(y_test, y_pred)
print(f"Validation error: {val_error}")

[0]	validation_0-rmse:596708.68460
[1]	validation_0-rmse:591295.72300
[2]	validation_0-rmse:585957.46317
[3]	validation_0-rmse:580680.43333
[4]	validation_0-rmse:575439.68551
[5]	validation_0-rmse:570280.15154
[6]	validation_0-rmse:565151.54487
[7]	validation_0-rmse:560083.70074
[8]	validation_0-rmse:555081.70275
[9]	validation_0-rmse:550125.15630
[10]	validation_0-rmse:545225.24631
[11]	validation_0-rmse:540374.71050
[12]	validation_0-rmse:535579.37825
[13]	validation_0-rmse:530834.89297
[14]	validation_0-rmse:526142.24349
[15]	validation_0-rmse:521496.96018
[16]	validation_0-rmse:516909.24518
[17]	validation_0-rmse:512360.93590
[18]	validation_0-rmse:507866.78170
[19]	validation_0-rmse:503419.47901
[20]	validation_0-rmse:499027.40229
[21]	validation_0-rmse:494685.64849
[22]	validation_0-rmse:490383.39808
[23]	validation_0-rmse:486126.77051
[24]	validation_0-rmse:481918.84922
[25]	validation_0-rmse:477762.99081
[26]	validation_0-rmse:473646.48054
[27]	validation_0-rmse:469577.35035
[2

[225]	validation_0-rmse:153731.01505
[226]	validation_0-rmse:153297.43881
[227]	validation_0-rmse:152926.51409
[228]	validation_0-rmse:152561.41225
[229]	validation_0-rmse:152193.28663
[230]	validation_0-rmse:151776.29929
[231]	validation_0-rmse:151424.47684
[232]	validation_0-rmse:151082.93718
[233]	validation_0-rmse:150723.81438
[234]	validation_0-rmse:150314.13107
[235]	validation_0-rmse:149959.40296
[236]	validation_0-rmse:149626.71822
[237]	validation_0-rmse:149212.64838
[238]	validation_0-rmse:148892.41969
[239]	validation_0-rmse:148525.39992
[240]	validation_0-rmse:148218.03558
[241]	validation_0-rmse:147902.42199
[242]	validation_0-rmse:147550.09904
[243]	validation_0-rmse:147249.49484
[244]	validation_0-rmse:146939.83067
[245]	validation_0-rmse:146602.41689
[246]	validation_0-rmse:146294.25350
[247]	validation_0-rmse:145958.81932
[248]	validation_0-rmse:145674.01872
[249]	validation_0-rmse:145392.79390
[250]	validation_0-rmse:145074.23979
[251]	validation_0-rmse:144782.18548
[

[447]	validation_0-rmse:114837.64627
[448]	validation_0-rmse:114785.17026
[449]	validation_0-rmse:114728.42778
[450]	validation_0-rmse:114573.93529
[451]	validation_0-rmse:114518.63553
[452]	validation_0-rmse:114471.80642
[453]	validation_0-rmse:114422.97865
[454]	validation_0-rmse:114344.35700
[455]	validation_0-rmse:114298.13183
[456]	validation_0-rmse:114182.92162
[457]	validation_0-rmse:114129.10485
[458]	validation_0-rmse:114096.89281
[459]	validation_0-rmse:114051.96395
[460]	validation_0-rmse:113939.44481
[461]	validation_0-rmse:113908.04345
[462]	validation_0-rmse:113797.41792
[463]	validation_0-rmse:113746.19632
[464]	validation_0-rmse:113715.57402
[465]	validation_0-rmse:113604.94535
[466]	validation_0-rmse:113548.84589
[467]	validation_0-rmse:113499.30936
[468]	validation_0-rmse:113469.64308
[469]	validation_0-rmse:113361.39957
[470]	validation_0-rmse:113313.48126
[471]	validation_0-rmse:113230.91753
[472]	validation_0-rmse:113190.83695
[473]	validation_0-rmse:113150.37844
[

[669]	validation_0-rmse:106721.20361
[670]	validation_0-rmse:106689.17405
[671]	validation_0-rmse:106635.00188
[672]	validation_0-rmse:106570.13197
[673]	validation_0-rmse:106547.93848
[674]	validation_0-rmse:106538.11931
[675]	validation_0-rmse:106498.93061
[676]	validation_0-rmse:106474.20106
[677]	validation_0-rmse:106466.65247
[678]	validation_0-rmse:106456.22553
[679]	validation_0-rmse:106425.34632
[680]	validation_0-rmse:106393.47633
[681]	validation_0-rmse:106351.10293
[682]	validation_0-rmse:106346.08208
[683]	validation_0-rmse:106310.71070
[684]	validation_0-rmse:106289.34882
[685]	validation_0-rmse:106264.90469
[686]	validation_0-rmse:106211.98255
[687]	validation_0-rmse:106199.20843
[688]	validation_0-rmse:106164.61439
[689]	validation_0-rmse:106153.07137
[690]	validation_0-rmse:106132.35957
[691]	validation_0-rmse:106090.11784
[692]	validation_0-rmse:106067.46410
[693]	validation_0-rmse:106029.25935
[694]	validation_0-rmse:106024.32859
[695]	validation_0-rmse:106011.69569
[

[891]	validation_0-rmse:100120.25209
[892]	validation_0-rmse:100072.38815
[893]	validation_0-rmse:100059.01088
[894]	validation_0-rmse:100052.89419
[895]	validation_0-rmse:100013.11773
[896]	validation_0-rmse:99976.16530
[897]	validation_0-rmse:99960.49184
[898]	validation_0-rmse:99950.81787
[899]	validation_0-rmse:99905.18660
[900]	validation_0-rmse:99905.46414
[901]	validation_0-rmse:99882.56683
[902]	validation_0-rmse:99854.40024
[903]	validation_0-rmse:99833.29602
[904]	validation_0-rmse:99810.85080
[905]	validation_0-rmse:99803.00278
[906]	validation_0-rmse:99769.44770
[907]	validation_0-rmse:99759.92221
[908]	validation_0-rmse:99720.53937
[909]	validation_0-rmse:99716.91155
[910]	validation_0-rmse:99697.61543
[911]	validation_0-rmse:99669.95265
[912]	validation_0-rmse:99657.14151
[913]	validation_0-rmse:99610.74790
[914]	validation_0-rmse:99607.23394
[915]	validation_0-rmse:99585.31293
[916]	validation_0-rmse:99576.07246
[917]	validation_0-rmse:99576.41486
[918]	validation_0-rmse

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 101)

In [45]:
xgb = xg.XGBRegressor(n_estimators = 428, max_depth = 3, random_state = 101, booster = 'gbtree', 
                      learning_rate = 0.01, enable_categorical=True)

# model validation
kf = KFold(n_splits = len(predictors))
results = cross_val_score(xgb, X, y, cv=kf)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

mse = {"train": [], "test": []}
mae = {"train": [], "test": []}
mape = {"train": [], "test": []}
r2 = {"train": [], "test": []}

for train, test in kf.split(X):
    X_train = X.iloc[train]
    X_test  = X.iloc[test]
    y_train = y[train]
    y_test  = y[test]

    # fit
    xgb.fit(X_train,y_train)

    # predict
    y_pred_train = xgb.predict(X_train)
    y_pred_test = xgb.predict(X_test)

    # assess
    mse["train"].append(mean_squared_error(y_train,y_pred_train))
    mse["test"].append(mean_squared_error(y_test,y_pred_test))
    
    mae["train"].append(mean_absolute_error(y_train,y_pred_train))
    mae["test"].append(mean_absolute_error(y_test,y_pred_test))

    mape["train"].append(mean_absolute_percentage_error(y_train,y_pred_train))
    mape["test"].append(mean_absolute_percentage_error(y_test,y_pred_test))

    r2["train"].append(r2_score(y_train,y_pred_train))
    r2["test"].append(r2_score(y_test,y_pred_test))

print("Train MSEs:", mse["train"])
print("Test MSEs :", mse["test"])
print("Train MSE :", np.mean(mse["train"]))
print("Test MSE  :", np.mean(mse["test"]))

print("Train MAEs:", mae["train"])
print("Test MAEs :", mae["test"])
print("Train MAE :", np.mean(mae["train"]))
print("Test MAE  :", np.mean(mae["test"]))

print("Train MAPEs:", mape["train"])
print("Test MAPEs :", mape["test"])
print("Train MAPE :", np.mean(mape["train"]))
print("Test MAPE  :", np.mean(mape["test"]))

print("Train R2s:", r2["train"])
print("Test R2s :", r2["test"])
print("Train R2 :", np.mean(r2["train"]))
print("Test R2  :", np.mean(r2["test"]))

Accuracy: 26.46% (72.26%)
Train MSEs: [10203075765.73917, 9115499135.86935, 10127095336.911139, 13641486977.45934, 13799244876.35584, 13811461695.604841, 13883002217.770535, 13893115448.924923, 12221378161.427872, 12650800100.037712]
Test MSEs : [60951563739.15353, 81687256190.50916, 61536068133.61095, 4074540654.3458, 3936705048.4745126, 3564654572.0079823, 2419429113.6803274, 1846024197.38884, 15664299099.677107, 22845361374.00294]
Train MSE : 12334615971.610071
Test MSE  : 25852590212.285114
Train MAEs: [67855.6974480051, 65152.40500318336, 68334.14025360781, 80236.30968803057, 80957.62259258276, 81330.86491603883, 81797.28148538616, 81988.14266390834, 74414.91884813283, 76720.78994602694]
Test MAEs : [199669.12655057252, 230408.36760496182, 199902.65660782444, 43377.25316078244, 48778.3528148855, 45049.79633826336, 40060.814680927346, 32004.48996175908, 87587.6360540153, 129201.1817041109]
Train MAE : 75878.81728449029
Test MAE  : 105603.96754781026
Train MAPEs: [0.1009636088958881

In [46]:
new_df = new_df.drop(columns = ['zipcode'])

In [47]:
X = new_df[predictors]
y = new_df["price"]

In [48]:
contin = predictors
predictors

['mortgageRates/thirtyYearFixedRate',
 'bathrooms',
 'adTargets/sqft',
 'bedrooms',
 'yearBuilt',
 'photoCount',
 'school_count',
 'closest_school_distance',
 'crime rate',
 'Median Household Income']

In [49]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
categorical_features = [#'zipcode'
    'homeType']  # Replace with your categorical feature names
continuous_features = [contin]  # Replace with your continuous feature names
# Preprocessing for categorical data
continuous_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('cont', continuous_transformer, continuous_features)
    ])

"""
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 101)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)
"""
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 101)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xg.XGBRegressor(n_estimators = 428, max_depth = 3, random_state = 101, booster = 'gbtree', 
                    learning_rate = 0.01, enable_categorical=True, 
                    eval_metric = "rmse", reg_lambda = 1, reg_alpha = 0.5,))
])
"""
pipeline.named_steps['model'].fit(
    pipeline.named_steps['preprocessor'].fit_transform(X_train), y_train,
    eval_set=[(pipeline.named_steps['preprocessor'].transform(X_val), y_val)],
    early_stopping_rounds=10,
    verbose=True
)
"""

"""
xgb = xg.XGBRegressor(n_estimators = 428, max_depth = 3, random_state = 101, booster = 'gbtree', 
                    learning_rate = 0.01, enable_categorical=True, 
                    eval_metric = "rmse", early_stopping_rounds=10, reg_lambda = 1, reg_alpha = 0.5,)

xgb.fit(X_train, y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=10,
            verbose=True
            )
"""
xgb.fit(X_train, y_train)
# model validation
kf = KFold(n_splits = len(predictors))
results = cross_val_score(xgb, X, y, cv=kf)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

mse = {"train": [], "test": []}
mae = {"train": [], "test": []}
mape = {"train": [], "test": []}
r2 = {"train": [], "test": []}

for train, test in kf.split(X):
    X_train = X.iloc[train]
    X_test  = X.iloc[test]
    y_train = y[train]
    y_test  = y[test]

    # fit
    xgb.fit(X_train,y_train)

    # predict
    y_pred_train = xgb.predict(X_train)
    y_pred_test = xgb.predict(X_test)

    # assess
    mse["train"].append(mean_squared_error(y_train,y_pred_train))
    mse["test"].append(mean_squared_error(y_test,y_pred_test))
    
    mae["train"].append(mean_absolute_error(y_train,y_pred_train))
    mae["test"].append(mean_absolute_error(y_test,y_pred_test))

    mape["train"].append(mean_absolute_percentage_error(y_train,y_pred_train))
    mape["test"].append(mean_absolute_percentage_error(y_test,y_pred_test))

    r2["train"].append(r2_score(y_train,y_pred_train))
    r2["test"].append(r2_score(y_test,y_pred_test))

print("Train MSEs:", mse["train"])
print("Test MSEs :", mse["test"])
print("Train MSE :", np.mean(mse["train"]))
print("Test MSE  :", np.mean(mse["test"]))

print("Train MAEs:", mae["train"])
print("Test MAEs :", mae["test"])
print("Train MAE :", np.mean(mae["train"]))
print("Test MAE  :", np.mean(mae["test"]))

print("Train MAPEs:", mape["train"])
print("Test MAPEs :", mape["test"])
print("Train MAPE :", np.mean(mape["train"]))
print("Test MAPE  :", np.mean(mape["test"]))

print("Train R2s:", r2["train"])
print("Test R2s :", r2["test"])
print("Train R2 :", np.mean(r2["train"]))
print("Test R2  :", np.mean(r2["test"]))

Accuracy: 26.71% (74.90%)
Train MSEs: [12398989601.052856, 10693660587.732704, 11334917200.303514, 15815150495.304422, 16190395090.889696, 16144885435.197302, 16334840514.558704, 16327882266.748077, 14439238005.327648, 14706987405.381123]
Test MSEs : [52769393441.64937, 82441645049.886, 60593789809.79978, 4265938810.7285194, 3923146016.745969, 2720148997.1269884, 2524632114.781909, 1875995150.954826, 18532659511.429306, 23539086494.684807]
Train MSE : 14438694660.249607
Test MSE  : 25318643539.778748
Train MAEs: [73977.29773052844, 70078.56489083722, 72121.00987836906, 86332.4074304966, 87390.7345673281, 87681.20024671052, 88303.09641549968, 88607.37838823468, 78936.47871578613, 81926.04427912158]
Test MAEs : [192186.0244513359, 224695.29460877864, 195569.26479007633, 43367.35406727099, 49573.45652433206, 41174.415851622136, 41172.35605879541, 34715.69018881453, 97860.1336042065, 130735.44060707458]
Train MAE : 81535.4212542912
Test MAE  : 105104.94307523072
Train MAPEs: [0.10764387698

In [50]:
new_df = pd.merge(new_df, zipcodes, on = 'price')


In [51]:
#zipcodes.dtypes


In [52]:
#new_df['zipcode'] = new_df['zipcode'].astype(str)
#new_df['zipcode'] = new_df['zipcode'].apply(lambda x: "zip: " + x )

In [53]:
#new_df  = new_df.drop(columns=['zipcode'])
#new_df.head()